In [ ]:
pip install pymongo

In [38]:
import pymongo
import bson
import json
from bson.code import Code
from bson.son import SON

In [39]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [47]:
print(myclient.list_database_names())

['admin', 'config', 'local', 'samples_pokemon']


In [46]:
#Q1 - Creating db amazon
mydb = myclient['amazon']
print("Database created........")

Database created........


In [42]:
print(mydb.list_collection_names())

[]


In [48]:
#Q2 - Creating collection reviews
mycol = mydb['reviews']
print("Collection created........")

Collection created........


In [49]:
# Reading and uploading each review in a seperate document list
document = []
for line in open('reviews_toys_and_games.json', 'r'):
    document.append(json.loads(line))

document

[{'reviewerID': 'A36NM32UFFE5BO',
  'asin': '048645195X',
  'reviewerName': 'michiganshopper',
  'helpful': [0, 0],
  'reviewText': 'This coloring book is one of my favorites! Sometimes it takes me a while to even get started because each page can be taken in so many colorways. Great adult or older child coloring book',
  'overall': 5.0,
  'summary': 'Wonderful.',
  'unixReviewTime': 1363046400,
  'reviewTime': '03 12, 2013'},
 {'reviewerID': 'A39EO6K0QCIL4N',
  'asin': '048645195X',
  'reviewerName': 'Micky',
  'helpful': [0, 0],
  'reviewText': 'I ordered a total of 4 design books and this is my favorite one of them all...love the finer lines and detail to each page...great for the pens I use to color them...do not recomend markers for most of the designs, it would be too difficult to stay in the lines...gel pens work perfectly!!!',
  'overall': 5.0,
  'summary': 'My favorite',
  'unixReviewTime': 1393545600,
  'reviewTime': '02 28, 2014'},
 {'reviewerID': 'A1G8DAO8QCAKGG',
  'asin':

In [50]:
try:
    x = mycol.insert_many(document) 
except:
    pass

In [51]:
#Q3 Creating a new collection "avg_scores"
mapFunction1 = Code("function() { emit(this.asin, this.overall) }")

reduceFunction1 = Code("function(key, value) { return Array.avg(value) }")

result = mydb['reviews'].map_reduce(mapFunction1, reduceFunction1, "avg_scores")

In [52]:
# Printing the first 100 entries of "avg_score" collection 
for document in mydb.avg_scores.find().limit(100):
    print(document)

{'_id': 'B00017IBT4', 'value': 4.3076923076923075}
{'_id': 'B000BRN8VA', 'value': 5.0}
{'_id': 'B000FTLGGS', 'value': 5.0}
{'_id': 'B000I03GZS', 'value': 3.0}
{'_id': 'B000CIU7A8', 'value': 4.016949152542373}
{'_id': 'B00001QEA6', 'value': 4.5}
{'_id': 'B0000ESZDC', 'value': 4.714285714285714}
{'_id': 'B000AOA998', 'value': 4.142857142857143}
{'_id': 'B000BXKTA2', 'value': 5.0}
{'_id': 'B000FA6HD4', 'value': 2.0}
{'_id': 'B00000ISJY', 'value': 5.0}
{'_id': 'B0006NGO4C', 'value': 3.0}
{'_id': 'B0009F3DVO', 'value': 3.5}
{'_id': 'B000F43IKU', 'value': 3.75}
{'_id': 'B000J42ZEG', 'value': 4.25}
{'_id': 'B00004U31Y', 'value': 4.5}
{'_id': 'B000GZCC1Y', 'value': 1.0}
{'_id': 'B00029PCLG', 'value': 5.0}
{'_id': 'B0009MASN8', 'value': 5.0}
{'_id': 'B000F8T4UE', 'value': 5.0}
{'_id': 'B00004YUYS', 'value': 3.962962962962963}
{'_id': 'B0007WJNXG', 'value': 4.333333333333333}
{'_id': 'B0006IRSHY', 'value': 5.0}
{'_id': 'B000065CMZ', 'value': 3.0}
{'_id': 'B0006N72T8', 'value': 4.4}
{'_id': 'B000

In [53]:
mapFunction2 = Code("function() {var value = {overall : this.overall, weight : this.helpful[1] + 1};emit(this.asin, value)}")
reduceFunction2 = Code("function(key, values) { reduceVal = { sum_pt: 0, tot_wgt: 0 };\
              for (var idx = 0; idx < values.length; idx++) {\
                  reduceVal.sum_pt+=  values[idx].overall*values[idx].weight;\
                  reduceVal.tot_wgt+= values[idx].weight;\
               }\
               weight_src = reduceVal.sum_pt/(reduceVal.tot_wgt +1);\
               return weight_src }")
result = mydb['reviews'].map_reduce(mapFunction2, reduceFunction2, "weighted_avg_scores")

In [54]:
# # Printing the first 100 entries of collection "weighted_avg_scores"
for document2 in mydb.weighted_avg_scores.find().limit(100):
    print(document2)

{'_id': '0924886722', 'value': 3.2}
{'_id': 'B0000Z17NO', 'value': 2.0}
{'_id': 'B000J55H9K', 'value': 2.5}
{'_id': 'B0006HB9F2', 'value': 4.166666666666667}
{'_id': 'B00004TQ44', 'value': 1.3333333333333333}
{'_id': 'B000ABD22M', 'value': 2.5}
{'_id': 'B000EJ9RHE', 'value': 2.0}
{'_id': 'B0002H7TV4', 'value': 4.589041095890411}
{'_id': 'B0006V56AM', 'value': 4.454545454545454}
{'_id': 'B000FBSMIQ', 'value': 0.5}
{'_id': 'B000EPFF74', 'value': 2.6666666666666665}
{'_id': 'B00005C1I3', 'value': 3.6}
{'_id': 'B0001NECIW', 'value': 4.0}
{'_id': 'B0006IEX7C', 'value': 4.691358024691358}
{'_id': 'B0001FR5H0', 'value': 4.26530612244898}
{'_id': 'B0007LYL5C', 'value': 3.4}
{'_id': 'B0006N6MT4', 'value': 4.0}
{'_id': 'B000BXKMGS', 'value': 4.285714285714286}
{'_id': 'B00006JUJR', 'value': 4.803030303030303}
{'_id': 'B0007UE8JC', 'value': 4.2153846153846155}
{'_id': 'B0006ZJ7RQ', 'value': 2.5}
{'_id': 'B0009XBY26', 'value': 4.2064516129032254}
{'_id': 'B0009XBZSO', 'value': 2.5}
{'_id': 'B000CE